In [299]:
#Import all relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import plotly.graph_objects as plo
import plotly.express as px
 
## This statement allows the visuals to render within your Jupyter Notebook.
%matplotlib inline

## Loading the data
We can now load the dataset into pandas using the read_csv() function. This converts the CSV file into a Pandas dataframe.

In [300]:
#Read in the csv file and convert to a Pandas dataframe
dfs: dict[str, pd.DataFrame] = {"raw": pd.read_csv("../data/the.csv")}
df = dfs["raw"].copy()

In [301]:
# for later
sdg_mapping = {
    1: "No Poverty",
    2: "Zero Hunger",
    3: "Good Health and Well-being",
    4: "Quality Education",
    5: "Gender Equality",
    6: "Clean Water and Sanitation",
    7: "Affordable and Clean Energy",
    8: "Decent Work and Economic Growth",
    9: "Industry, Innovation, and Infrastructure",
    10: "Reduced Inequalities",
    11: "Sustainable Cities and Communities",
    12: "Responsible Consumption and Production",
    13: "Climate Action",
    14: "Life Below Water",
    15: "Life On Land",
    16: "Peace, Justice, and Strong Institutions",
    17: "Partnerships for the Goals"
}

### Viewing the dataframe
We can get a quick sense of the size of our dataset by using the shape method. This returns a tuple with the number of rows and columns in the dataset.

In [302]:
display(df.head(2))

,Rank,University Name,ranking-institution-title href,Location,sdg-score-multi__number,sdg-score-multi__value,sdg-score-multi__number 2,sdg-score-multi__value 2,sdg-score-multi__number 3,sdg-score-multi__value 3,sdg-score-multi__value 4,scores,sdg-score-multi__number 5,sdg-score-multi__value 5,sdg-score-multi__number 6,sdg-score-multi__value 6,sdg-score-multi__number 7,sdg-score-multi__value 7,sdg-score-multi__number 8,sdg-score-multi__value 8
0,1,Western Sydney University,https://www.timeshighereducation.com/world-uni...,Australia,5.0,80.3,12.0,93.4,15.0,96.7,98.8,99.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,University of Manchester,https://www.timeshighereducation.com/world-uni...,United Kingdom,15.0,97.4,12.0,92.9,11.0,92.2,94.8,97.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**NOTE:** We have null cells and meaningless columns naming

## Dataset info

| Number | Meaning                                  |
| ------ | ---------------------------------------- |
| 1      | No Poverty                               |
| 2      | Zero Hunger                              |
| 3      | Good Health and Well-being               |
| 4      | Quality Education                        |
| 5      | Gender Equality                          |
| 6      | Clean Water and Sanitation               |
| 7      | Affordable and Clean Energy              |
| 8      | Decent Work and Economic Growth          |
| 9      | Industry, Innovation, and Infrastructure |
| 10     | Reduced Inequalities                     |
| 11     | Sustainable Cities and Communities       |
| 12     | Responsible Consumption and Production   |
| 13     | Climate Action                           |
| 14     | Life Below Water                         |
| 15     | Life On Land                             |
| 16     | Peace, Justice, and Strong Institutions  |
| 17     | Partnerships for the Goals               |

The Times Higher Education Impact Rankings are the only global performance tables that assess universities against the United Nations’ Sustainable Development Goals (SDGs). sdg-score-multi__numbers will be replaced using the table above

## 1. Data Profiling:
Data profiling is a comprehensive process of examining the data available in an existing dataset and collecting statistics and information about that data. 

In [303]:
df.info()
print("\nshape:", df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Rank                            1591 non-null   object 
 1   University Name                 1591 non-null   object 
 2   ranking-institution-title href  1500 non-null   object 
 3   Location                        1500 non-null   object 
 4   sdg-score-multi__number         867 non-null    float64
 5   sdg-score-multi__value          867 non-null    object 
 6   sdg-score-multi__number 2       826 non-null    float64
 7   sdg-score-multi__value 2        825 non-null    object 
 8   sdg-score-multi__number 3       812 non-null    float64
 9   sdg-score-multi__value 3        805 non-null    object 
 10  sdg-score-multi__value 4        1583 non-null   object 
 11  scores                          1591 non-null   object 
 12  sdg-score-multi__number 5       56

<p style="color: red">We can explain the nulls by referencing the table</p>

The process of profiling differs slightly for categorical and numerical variables due to their inherent differences.

**The two main types of data are:**
- Quantitative (numerical) data
- Qualitative (categorical) data

### Data Quality Checks
Data quality checks involve the process of ensuring that the data is accurate, complete, consistent, relevant, and reliable. 


**Here are typical steps involved in checking data quality:**

#### 1. Reliability:
Evaluate the data's source and collection process to determine its trustworthiness.

<a href="https://www.timeshighereducation.com/impactrankings">Times Higher Education Impact Rankings 2023</a>

#### 2. Timeliness: 
Ensure the data is up-to-date and reflective of the current situation or the period of interest for the analysis.

<p style="color: red">The data is up to date for our case</p>

#### 3. Consistency: 

Confirm that the data is consistent within the dataset and across multiple data sources. For example, the same data point should not have different values in different places.


<p style="color: red">Columns names are meaningless and should be renamed</p>

In [304]:
# Renaming columns
new_column_names = {}
for col in df.columns:
    if 'sdg-score-multi__number' in col:
        category_num = col.split()[-1] if len(col.split()) > 1 else ""
        new_column_names[col] = f'sdg_category{category_num}'
    elif 'sdg-score-multi__value' in col:
        value_num = col.split()[-1] if len(col.split()) > 1 else ""
        new_column_names[col] = f'sdg_value{value_num}'

new_column_names["ranking-institution-title href"] = "href"
new_column_names["Rank"] = "rank"
new_column_names["University Name"] = "university_name"
new_column_names["Location"] = "location"

df.rename(columns=new_column_names, inplace=True)
dfs["cleaned_raw"] = df.copy()
# Display the updated DataFrame
display(df)

,rank,university_name,href,location,sdg_category,sdg_value,sdg_category2,sdg_value2,sdg_category3,sdg_value3,sdg_value4,scores,sdg_category5,sdg_value5,sdg_category6,sdg_value6,sdg_category7,sdg_value7,sdg_category8,sdg_value8
0,1,Western Sydney University,https://www.timeshighereducation.com/world-uni...,Australia,5.0,80.3,12.0,93.4,15.0,96.7,98.8,99.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,University of Manchester,https://www.timeshighereducation.com/world-uni...,United Kingdom,15.0,97.4,12.0,92.9,11.0,92.2,94.8,97.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Queen’s University,https://www.timeshighereducation.com/world-uni...,Canada,2.0,90.6,16.0,93.4,11.0,91.0,92.6,97.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Universiti Sains Malaysia,https://www.timeshighereducation.com/world-uni...,Malaysia,16.0,94.1,2.0,86.4,1.0,84.9,97.8,96.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,University of Tasmania,https://www.timeshighereducation.com/world-uni...,Australia,13.0,92.8,15.0,96.2,14.0,95.1,96.3,96.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,1001+,Zarqa University,https://www.timeshighereducation.com/world-uni...,Jordan,NaN,NaN,NaN,NaN,1.0,23.3–38.1,1.5–45.2,7.9–53.8,NaN,NaN,4.0,43.6–50.9,3.0,32.1–44.6,NaN,NaN
1587,1001+,Zewail City of Science and Technology,https://www.timeshighereducation.com/world-uni...,Egypt,3.0,44.7–54.5,7.0,11.1–38.6,NaN,NaN,45.3–53.3,7.9–53.8,1.0,54.5–60.1,NaN,NaN,NaN,NaN,NaN,NaN
1588,1001+,Zhytomyr Ivan Franko State University,https://www.timeshighereducation.com/world-uni...,Ukraine,NaN,NaN,10.0,3.9–25.5,NaN,NaN,1.5–45.2,7.9–53.8,NaN,NaN,8.0,35.7–48.3,5.0,43.7–51.5,NaN,NaN
1589,1001+,University of Žilina,https://www.timeshighereducation.com/world-uni...,Slovakia,11.0,20.9–38.4,NaN,NaN,NaN,NaN,1.5–45.2,7.9–53.8,NaN,NaN,9.0,17.2–33.8,NaN,NaN,4.0,2.8–35.7


#### 4. Relevance: 
Assess whether the data is appropriate and applicable for the intended analysis. Data that is not relevant can skew results and lead to incorrect conclusions.

**Key considerations for relevance include:**

> 1. Sample Appropriateness: Confirm that your data sample aligns with your analysis objectives. For instance, utilizing data from the Northern region will not yield accurate insights for the Western region of the Kingdom.
>
> 2. Variable Selection: Any column will not be relevant for our analysis, we can get rid of these using the drop() method. We will set the “axis” argument to 1 since we’re dealing with columns, and set the “inplace” argument to True to make the change permanent.


In [305]:
df.drop(["href"], inplace=True, axis=1)

In [306]:
df.head(2)

,rank,university_name,location,sdg_category,sdg_value,sdg_category2,sdg_value2,sdg_category3,sdg_value3,sdg_value4,scores,sdg_category5,sdg_value5,sdg_category6,sdg_value6,sdg_category7,sdg_value7,sdg_category8,sdg_value8
0,1,Western Sydney University,Australia,5.0,80.3,12.0,93.4,15.0,96.7,98.8,99.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,University of Manchester,United Kingdom,15.0,97.4,12.0,92.9,11.0,92.2,94.8,97.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 5. Uniqueness: 
Check for and remove duplicate records to prevent skewed analysis results.


In [307]:
print(f"Number of duplicate: {df.duplicated().sum()}")

Number of duplicate: 0


In [308]:
# go to delete duplicates columns

#### 6. Completeness: 
Ensure that no critical data is missing. This might mean checking for null values or required fields that are empty.

We will start by checking the dataset for missing or null values. For this, we can use the isna() method which returns a dataframe of boolean values indicating if a field is null or not. To group all missing values by column, we can include the sum() method.

In [309]:
#Display number missing values per column
print(f"Is null: \n{df.isna().sum()}\n\n")

Is null: 
rank                  0
university_name       0
location             91
sdg_category        724
sdg_value           724
sdg_category2       765
sdg_value2          766
sdg_category3       779
sdg_value3          786
sdg_value4            8
scores                0
sdg_category5      1026
sdg_value5         1034
sdg_category6      1093
sdg_value6         1095
sdg_category7      1152
sdg_value7         1152
sdg_category8      1108
sdg_value8         1108
dtype: int64




<p style="color: red">Here the nulls are result of expanding the data on multiple columns</p>

In [310]:
def consolidate_sdgs(row: pd.Series) -> pd.DataFrame:
    categories = []
    values = []

    # Collect valid SDG categories and values
    for i in range(1, 9):
        if i == 1:
            category_key = f'sdg_category'
            value_key = f'sdg_value'
        else:
            category_key = f'sdg_category{i}'
            value_key = f'sdg_value{i}'

        if category_key in row and value_key in row and pd.notna(row[category_key]) and pd.notna(row[value_key]):
            categories.append(str(int(row[category_key])))
            values.append(row[value_key])

    # Ensure there are at least three SDGs before adding the fourth
    while len(categories) < 3:
        categories.append(None)
    while len(values) < 3:
        values.append(None)

    # Fourth SDG is fixed to category 17 with its predefined value
    categories.append(17)
    values.append(row['sdg_value4'])

    # Combine the required information in a series
    return pd.Series([
        row['rank'], row['university_name'], row['location'],
        categories[0], values[0], categories[1], values[1], categories[2], values[2],
        categories[3], values[3], row['scores']
    ], index=[
        'rank', 'university_name','location',
        'first_sdg', 'first_sdg_value', 'second_sdg', 'second_sdg_value',
        'third_sdg', 'third_sdg_value', 'fourth_sdg', 'fourth_sdg_value', 'score'
    ])


# Example DataFrame and application of the function
df_consolidated = df.apply(consolidate_sdgs, axis=1)
dfs["columns_fixed"] =  df_consolidated.copy()
df = df_consolidated.copy()
display(df)

,rank,university_name,location,first_sdg,first_sdg_value,second_sdg,second_sdg_value,third_sdg,third_sdg_value,fourth_sdg,fourth_sdg_value,score
0,1,Western Sydney University,Australia,5,80.3,12,93.4,15,96.7,17,98.8,99.4
1,2,University of Manchester,United Kingdom,15,97.4,12,92.9,11,92.2,17,94.8,97.5
2,3,Queen’s University,Canada,2,90.6,16,93.4,11,91.0,17,92.6,97.2
3,4,Universiti Sains Malaysia,Malaysia,16,94.1,2,86.4,1,84.9,17,97.8,96.9
4,5,University of Tasmania,Australia,13,92.8,15,96.2,14,95.1,17,96.3,96.6
...,...,...,...,...,...,...,...,...,...,...,...,...
1586,1001+,Zarqa University,Jordan,1,23.3–38.1,4,43.6–50.9,3,32.1–44.6,17,1.5–45.2,7.9–53.8
1587,1001+,Zewail City of Science and Technology,Egypt,3,44.7–54.5,7,11.1–38.6,1,54.5–60.1,17,45.3–53.3,7.9–53.8
1588,1001+,Zhytomyr Ivan Franko State University,Ukraine,10,3.9–25.5,8,35.7–48.3,5,43.7–51.5,17,1.5–45.2,7.9–53.8
1589,1001+,University of Žilina,Slovakia,11,20.9–38.4,9,17.2–33.8,4,2.8–35.7,17,1.5–45.2,7.9–53.8


In [311]:
print(f"Is null: \n{df.isna().sum()}\n\n")

Is null: 
rank                  0
university_name       0
location             91
first_sdg             0
first_sdg_value       0
second_sdg            9
second_sdg_value      9
third_sdg           298
third_sdg_value     298
fourth_sdg            0
fourth_sdg_value      8
score                 0
dtype: int64




In [312]:
df[df.isnull().any(axis=1)]

,rank,university_name,location,first_sdg,first_sdg_value,second_sdg,second_sdg_value,third_sdg,third_sdg_value,fourth_sdg,fourth_sdg_value,score
103,101–200,Ahlia University,Bahrain,10,86.4,8,73.3,None,None,17,81.9–86.9,82.2–88.2
221,201–300,University of Central Lancashire,United Kingdom,12,66.8–76.6,3,73.5–78.9,None,None,17,75.6–81.7,76.8–82.1
227,201–300,Cranfield University,United Kingdom,8,73.2,16,70.0–77.0,None,None,17,93.3,76.8–82.1
236,201–300,Frederick University,Cyprus,4,77.5,7,67.9,None,None,17,75.6–81.7,76.8–82.1
245,201–300,IMT Mines Alès,France,12,78.8,8,71.7,None,None,17,81.9–86.9,76.8–82.1
...,...,...,...,...,...,...,...,...,...,...,...,...
1570,1001+,West University of Timişoara,Romania,10,40.0–51.7,5,51.6–56.0,None,None,17,1.5–45.2,7.9–53.8
1571,1001+,West Ukrainian National University,Ukraine,8,62.4–66.3,10,51.8–57.5,None,None,17,1.5–45.2,7.9–53.8
1573,1001+,University of Wisconsin-Eau Claire,United States,10,65.2–73.3,3,54.7–63.8,None,None,17,1.5–45.2,7.9–53.8
1579,1001+,University of Wrocław,Poland,15,47.8–59.4,8,48.4–57.7,None,None,17,1.5–45.2,7.9–53.8


<p style="color: red">Useless Data will be dropped with errors can't be solved</p>

In [313]:
# go to clean them 

#### 7. Check Accuracy:

Verify that the data is correct and precise. This could involve comparing data samples with known sources or using validation rules.

**The process includes:**
1. Validating the appropriateness of data types for the dataset.
2. Identifying outliers  using established validation  rule

In [314]:
# check columns types
df.dtypes

rank                object
university_name     object
location            object
first_sdg           object
first_sdg_value     object
second_sdg          object
second_sdg_value    object
third_sdg           object
third_sdg_value     object
fourth_sdg           int64
fourth_sdg_value    object
score               object
dtype: object

In [315]:
# go to clean them

In [316]:
# check outliers

**What is an Outlier?** 
Outlier is an row/observation that appears far away and diverges from an overall pattern in a sample.

**What are the types of Outliers?**
1. Univariate: These outliers can be found when we look at distribution of a single variable
2. Multivariate: are outliers in an n-dimensional space. In order to find them, you have to look at distributions in multi-dimensions. example (hight=100, weight=100) for a person

**What causes Outliers?**
Whenever we come across outliers, the ideal way to tackle them is to find out the reason of having these outliers. The method to deal with them would then depend on the reason of their occurrence.

Let’s understand various types of outliers:

1. Data Entry Errors:- Human errors such as errors caused during data collection, recording, or entry can cause outliers in data.
2. Measurement Error: It is the most common source of outliers. This is caused when the measurement instrument used turns out to be faulty.
3. Data Processing Error: Whenever we perform data mining, we extract data from multiple sources. It is possible that some manipulation or extraction errors may lead to outliers in the dataset.
4. Sampling error: For instance, we have to measure the height of athletes. By mistake, we include a few basketball players in the sample. This inclusion is likely to cause outliers in the dataset.
5. Natural Outlier: When an outlier is not artificial (due to error), it is a natural outlier. For instance: In my last assignment with one of the renowned insurance company, I noticed that the performance of top 50 financial advisors was far higher than rest of the population. Surprisingly, it was not due to any error. Hence, whenever we perform any data mining activity with advisors, we used to treat this segment separately.


**What is the impact of Outliers on a dataset?**


![image.png](https://www.analyticsvidhya.com/wp-content/uploads/2015/02/Outlier_31.png)



**How to detect Outliers?**

1. Most commonly used method to detect outliers is visualization (Univariate Graphical Analysis).

We use 3 common visualization methods:
>- Box-plot: A box plot is a method for graphically depicting groups of numerical data through their quartiles. The box extends from the Q1 to Q3 quartile values of the data, with a line at the median (Q2). The whiskers extend from the edges of the box to show the range of the data. Outlier points are those past the end of the whiskers. Box plots show robust measures of location and spread as well as providing information about symmetry and outliers.
>
>  
>![image.png](https://miro.medium.com/v2/resize:fit:698/format:webp/1*VK5iHA2AB28HSZwWwUbNYg.png)
>
>
>- Histogram
>- Scatter Plot: A scatter plot is a mathematical diagram using Cartesian coordinates to display values for two variables for a set of data. The data are displayed as a collection of points, each having the value of one variable determining the position on the horizontal axis and the value of the other variable determining the position on the vertical axis. The points that are far from the population can be termed as an outlier.
>
>  
>![image.png](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*Ov6aH-8yIwNoUxtMFwgx4g.png)
>
>

2. Using statistical method (Univariate Non-Graphical analysis):
>- Any value, which is beyond the range of -1.5 x IQR to 1.5 x IQR
 
![image.png](https://www.whatissixsigma.net/wp-content/uploads/2015/07/Box-Plot-Diagram-to-identify-Outliers-figure-1.png)

>- Use capping methods. Any value which out of range of 5th and 95th percentile can be considered as outlier
>- Data points, three or more standard deviation away from mean are considered outlier: The Z-score is the signed number of standard deviations by which the value of an observation or data point is above the mean value of what is being observed or measured. While calculating the Z-score we re-scale and center the data and look for data points that are too far from zero. These data points which are way too far from zero will be treated as the outliers. In most of the cases, a threshold of 3 or -3 is used i.e if the Z-score value is greater than or less than 3 or -3 respectively, that data point will be identified as outliers.
> - Outlier detection is merely a special case of the examination of data for influential data points and it also depends on the business understanding


In [317]:
# go to univariate graphical analysis
# go to lesson : data visualisation 1 - chart type section
# then go to univariate graphical analysis
# detect outliers using graphs varbaly

In [318]:
# go to lesson: statistics 1 then statistics 3
# then go to univariate Non graphical analysis
# detect outliers using numerical statistics 

In [319]:
# go to delete ouliers

## 2. Data Cleaning: 

Preliminary findings from data profiling can lead to cleaning the data by:
- Handling missing values
- Correcting errors.
- Dealing with outliers.

-------------------



### Handling missing values:

**Why my data has missing values?**
They may occur at two stages:
1. Data Extraction: It is possible that there are problems with extraction process. Errors at data extraction stage are typically easy to find and can be corrected easily as well.
2. Data collection: These errors occur at time of data collection and are harder to correct.

**Why do we need to handle the missing data?**
To avoid:
- Bias the conclusions.
- Leading the business to make wrong decisions.

**Which are the methods to treat missing values ?**
1. Deletion: we delete rows where any of the variable is missing. Simplicity is one of the major advantage of this method, but this method reduces the power of model because it reduces the sample size.

2. Imputation: is a method to fill in the missing values with estimated ones. This imputation is one of the most frequently used methods.

    2.1. Mean/ Mode/ Median Imputation: It consists of replacing the missing data for a given attribute by the mean or median (quantitative attribute) or mode (qualitative attribute) of all known values of that variable.
    > It can be of two types:
    > - Generalized Imputation: In this case, we calculate the mean or median for all non missing values of that variable then replace missing value with mean or median.
    > - Similar case Imputation: In this case, we calculate average for each group individually of non missing values then replace the missing value based on the group.

    2.2. Constant Value
   
    2.3. Forward Filling
   
    2.4. Backward Filling

6. Prediction Model:  Prediction model is one of the sophisticated method for handling missing data. Here, we create a predictive model to estimate values that will substitute the missing data.  In this case, we divide our data set into two sets: One set with no missing values for the variable and another one with missing values. First data set become training data set of the model while second data set with missing values is test data set and variable with missing values is treated as target variable. Next, we create a model to predict target variable based on other attributes of the training data set and populate missing values of test data set.

> There are 2 drawbacks for this approach:
> - The model estimated values are usually more well-behaved than the true values
> - If there are no relationships with attributes in the data set and the attribute with missing values, then the model will not be precise for estimating missing values.

9. KNN Imputation: In this method of imputation, the missing values of an attribute are imputed using the given number of attributes that are most similar to the attribute whose values are missing. The similarity of two attributes is determined using a distance function. It is also known to have certain advantage & disadvantages.

   > **Advantages:**
   > - k-nearest neighbour can predict both qualitative & quantitative attributes
   > - Creation of predictive model for each attribute with missing data is not required
   > - Attributes with multiple missing values can be easily treated
   > - Correlation structure of the data is taken into consideration

   > **Disadvantage:**
   > - KNN algorithm is very time-consuming in analyzing large database. It searches through all the dataset looking for the most similar instances.
   > - Choice of k-value is very critical. Higher value of k would include attributes which are significantly different from what we need whereas lower value of k implies missing out of significant attributes.

--------------------


In [320]:
print(f"Is null: \n{df.isna().sum()}\n\n")

Is null: 
rank                  0
university_name       0
location             91
first_sdg             0
first_sdg_value       0
second_sdg            9
second_sdg_value      9
third_sdg           298
third_sdg_value     298
fourth_sdg            0
fourth_sdg_value      8
score                 0
dtype: int64




<p style="color: red">Useless Data will be dropped with errors can't be solved</p>

This will be fixed later

### Correcting errors

-------------------

In [321]:
df= df.dropna(axis=0)
print(f"Is null: \n{df.isna().sum()}\n\n")

Is null: 
rank                0
university_name     0
location            0
first_sdg           0
first_sdg_value     0
second_sdg          0
second_sdg_value    0
third_sdg           0
third_sdg_value     0
fourth_sdg          0
fourth_sdg_value    0
score               0
dtype: int64




In [322]:
df.dtypes

rank                object
university_name     object
location            object
first_sdg           object
first_sdg_value     object
second_sdg          object
second_sdg_value    object
third_sdg           object
third_sdg_value     object
fourth_sdg           int64
fourth_sdg_value    object
score               object
dtype: object

In [323]:
# # n_sdg refer to a category
# df["second_sdg"] = df["second_sdg"].astype(int)
# df["third_sdg"] = df["third_sdg"].astype(int)
# df["fourth_sdg"] = df["fourth_sdg"].astype(int)
# df.dtypes

In [324]:
df["rank"].value_counts()

rank
1001+       407
401–600     160
601–800     153
801–1000    132
101–200      98
           ... 
39            1
2             1
56            1
57            1
55            1
Name: count, Length: 64, dtype: int64

<p style="color: red">Fix data type issues</p>

In [325]:
# Function to calculate mean if there's a hyphen
def calculate_mean_if_range(value):
    if '–' in str(value):
        low, high = value.split('–')
        return (float(low) + float(high)) / 2
    return float(value)

# Apply the function to the 'score' column
df['score'] = df['score'].apply(calculate_mean_if_range)

# Print the modified DataFrame to verify the changes
display(df)

,rank,university_name,location,first_sdg,first_sdg_value,second_sdg,second_sdg_value,third_sdg,third_sdg_value,fourth_sdg,fourth_sdg_value,score
0,1,Western Sydney University,Australia,5,80.3,12,93.4,15,96.7,17,98.8,99.40
1,2,University of Manchester,United Kingdom,15,97.4,12,92.9,11,92.2,17,94.8,97.50
2,3,Queen’s University,Canada,2,90.6,16,93.4,11,91.0,17,92.6,97.20
3,4,Universiti Sains Malaysia,Malaysia,16,94.1,2,86.4,1,84.9,17,97.8,96.90
4,5,University of Tasmania,Australia,13,92.8,15,96.2,14,95.1,17,96.3,96.60
...,...,...,...,...,...,...,...,...,...,...,...,...
1586,1001+,Zarqa University,Jordan,1,23.3–38.1,4,43.6–50.9,3,32.1–44.6,17,1.5–45.2,30.85
1587,1001+,Zewail City of Science and Technology,Egypt,3,44.7–54.5,7,11.1–38.6,1,54.5–60.1,17,45.3–53.3,30.85
1588,1001+,Zhytomyr Ivan Franko State University,Ukraine,10,3.9–25.5,8,35.7–48.3,5,43.7–51.5,17,1.5–45.2,30.85
1589,1001+,University of Žilina,Slovakia,11,20.9–38.4,9,17.2–33.8,4,2.8–35.7,17,1.5–45.2,30.85


Sort the rank using the new Score

In [326]:
# Sort the DataFrame by the 'score' column in descending order
df = df.sort_values(by='score', ascending=False)

# Reset 'rank' based on new sorting
df['rank'] = range(1, len(df) + 1)

# Print the modified DataFrame to verify the changes
display(df)

,rank,university_name,location,first_sdg,first_sdg_value,second_sdg,second_sdg_value,third_sdg,third_sdg_value,fourth_sdg,fourth_sdg_value,score
0,1,Western Sydney University,Australia,5,80.3,12,93.4,15,96.7,17,98.8,99.40
1,2,University of Manchester,United Kingdom,15,97.4,12,92.9,11,92.2,17,94.8,97.50
2,3,Queen’s University,Canada,2,90.6,16,93.4,11,91.0,17,92.6,97.20
3,4,Universiti Sains Malaysia,Malaysia,16,94.1,2,86.4,1,84.9,17,97.8,96.90
4,5,University of Tasmania,Australia,13,92.8,15,96.2,14,95.1,17,96.3,96.60
...,...,...,...,...,...,...,...,...,...,...,...,...
1183,1220,"University of Finance and Economics, Mongolia",Mongolia,8,57.8–62.2,16,45.2–58.6,5,34.4–43.6,17,1.5–45.2,30.85
1182,1221,Fergana State University,Uzbekistan,10,25.6–39.9,2,38.4–48.2,4,43.6–50.9,17,1.5–45.2,30.85
1181,1222,Fergana Polytechnic Institute,Uzbekistan,7,38.7–51.8,8,35.7–48.3,4,35.8–43.5,17,1.5–45.2,30.85
1180,1223,Fergana Medical Institute of Public Health,Uzbekistan,1,38.2–49.4,4,2.8–35.7,3,54.7–63.8,17,1.5–45.2,30.85


In [327]:
def calculate_mean_if_range(value: str) -> float:
    if '–' in value:
        low, high = value.split('–')
        return (float(low) + float(high)) / 2

    return float(value)

# Identify all columns that might contain range values and need processing
value_columns = ["first_sdg_value","second_sdg_value","third_sdg_value","fourth_sdg_value"]

# Apply the function to each of these columns
for column in value_columns:
    df[column] = df[column].map(calculate_mean_if_range)


In [328]:
# Print the modified DataFrame to verify the changes
display(df)

,rank,university_name,location,first_sdg,first_sdg_value,second_sdg,second_sdg_value,third_sdg,third_sdg_value,fourth_sdg,fourth_sdg_value,score
0,1,Western Sydney University,Australia,5,80.30,12,93.40,15,96.70,17,98.80,99.40
1,2,University of Manchester,United Kingdom,15,97.40,12,92.90,11,92.20,17,94.80,97.50
2,3,Queen’s University,Canada,2,90.60,16,93.40,11,91.00,17,92.60,97.20
3,4,Universiti Sains Malaysia,Malaysia,16,94.10,2,86.40,1,84.90,17,97.80,96.90
4,5,University of Tasmania,Australia,13,92.80,15,96.20,14,95.10,17,96.30,96.60
...,...,...,...,...,...,...,...,...,...,...,...,...
1183,1220,"University of Finance and Economics, Mongolia",Mongolia,8,60.00,16,51.90,5,39.00,17,23.35,30.85
1182,1221,Fergana State University,Uzbekistan,10,32.75,2,43.30,4,47.25,17,23.35,30.85
1181,1222,Fergana Polytechnic Institute,Uzbekistan,7,45.25,8,42.00,4,39.65,17,23.35,30.85
1180,1223,Fergana Medical Institute of Public Health,Uzbekistan,1,43.80,4,19.25,3,59.25,17,23.35,30.85


In [329]:
# go back to 7th dimension Accuracy 

### Dealing with outliers:

**How to remove Outliers?**
Most of the ways to deal with outliers are similar to the methods of missing values like deleting rows, transforming them, binning them, treat them as a separate group, imputing values and other statistical methods. Here, we will discuss the common techniques used to deal with outliers:

1. Deleting rows: We delete outlier values if it is due to data entry error, data processing error or outlier rows are very small in numbers. We can also use trimming at both ends to remove outliers.

2. Imputing: Like imputation of missing values, we can also impute outliers. We can use mean, median, mode imputation methods. Before imputing values, we should analyse if it is natural outlier or artificial. If it is artificial, we can go with imputing values. We can also use statistical model to predict values of outlier rows and after that we can impute it with predicted values.

3. Treat separately: If there are significant number of outliers, we should treat them separately in the statistical model. One of the approach is to treat both groups as two different groups and build individual model for both groups and then combine the output.


In [336]:
for col in ["second_sdg_value","third_sdg_value", "fourth_sdg_value", "rank", "location", "score"]:
    fig = plo.Figure(plo.Box(y=df[col], name=col))
    fig.show()

<p style="color: red">We can see that there is no outliers, except in location which is normal</p>

## 3. Univariate Analysis: 

This involves examining single variables to understand their characteristics (distribution, central tendency, dispersion, and shape).

We calculate **numerical values** about the data that tells us about the distribution of the data. We also **draw graphs** showing visually how the data is distributed. **To answer the following questions about Features/characteristics of Data:**
- Where is the center of the data? (location)
- How much does the data vary? (scale)
- What is the shape of the data? (shape)

**The benefits of this analysis:**
Statistics summary gives a high-level idea to identify whether the data has any outliers, data entry error, distribution of data such as the data is normally distributed or left/right skewed

**In this step, we will explore variables one by one using following approaches:**

### 1. Univariate Graphical Analysis:
Method to perform uni-variate analysis will depend on whether the variable type is categorical or numerical.

In [331]:
# Histogram of first_sdg_value
fig_first_sdg_value = px.histogram(df, x='first_sdg_value', nbins=20, title='Distribution of First SDG Values')
fig_first_sdg_value.show()

# Histogram of score
fig_score = px.histogram(df, x='score', nbins=20, title='Distribution of Scores')
fig_score.show()



The outlier is clear in Distribution of Scores

#### I. Categorical Variables:

we’ll use frequency table to understand distribution of each category
- Bar Chart (Ordinal) - Orderd
- Pie Chart (Nominal) - non Orderd

In [332]:
# Assuming df is your DataFrame and 'location' counts have been calculated
location_counts = df.nlargest(25, "score")['location'].value_counts().reset_index()
location_counts.columns = ['country', 'count']  # Rename columns for clarity

fig_location = px.bar(location_counts, x='country', y='count',
                      labels={'country': 'Country', 'count': 'Count'},
                      title='Frequency of Top 25 Universities by Country')
fig_location.show()

#### II. Numerical Variables:

we need to understand the central tendency and spread of the variable (Descriptive Analysis) using:
   - Box plot
   - Histogram

In [333]:
# Histogram for 'first_sdg_value'
fig_histogram = px.histogram(df, x='first_sdg_value', nbins=10,
                             title='Distribution of First SDG Values')
fig_histogram.show()

# Scatter Plot to examine the relationship between 'first_sdg_value' and 'score'
fig_scatter = px.scatter(df.head(500), x='first_sdg_value', y='score',
                         title='Relationship between First SDG Value and Score',
                         labels={'first_sdg_value': 'First SDG Value', 'score': 'Score'})
fig_scatter.show()

print("Here we can see the effect of duplicated scores on the relations")

# Box Plot for SDG values
fig_box = px.box(df, y=['first_sdg_value', 'second_sdg_value', 'third_sdg_value', 'fourth_sdg_value'],
                 title='Distribution of SDG Values across Categories')
fig_box.show()

print("Also, here we can see the effect of duplicated fourth_value on the relations")

Here we can see the effect of duplicated scores on the relations


Also, here we can see the effect of duplicated fourth_value on the relations


### 2. Univariate Non-Graphical analysis: 

- Where is the center of the data? (location) --> **Measures of central tendency**
- How much does the data vary? (scale) --> **Measure of variability**
- What is the shape of the data? (shape) --> **Measures of variation combined with an average (measure of center) gives a good picture of the distribution of the data.**

In [334]:
# Select only the numeric columns for analysis
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Print basic statistical summaries for numerical columns
print("Summary Statistics:")
print(numeric_df.describe().T)

# Additional specific statistics
print("\nMedian of each numeric column:")
print(numeric_df.median())

print("\nMode of each numeric column:")
try:
    print(numeric_df.mode().iloc[0])
except IndexError:
    print("No mode found.")

print("\nVariance of each numeric column:")
print(numeric_df.var())

print("\nRange of each numeric column:")
print(numeric_df.max() - numeric_df.min())

Summary Statistics:
                   count        mean         std    min       25%     50%  \
rank              1224.0  612.500000  353.482673   1.00  306.7500  612.50   
first_sdg_value   1224.0   60.436438   19.925553  11.15   47.6000   62.15   
second_sdg_value  1224.0   58.896855   18.700488  11.15   47.5125   61.15   
third_sdg_value   1224.0   58.262173   18.474208   5.60   47.2500   60.00   
fourth_sdg        1224.0   17.000000    0.000000  17.00   17.0000   17.00   
fourth_sdg_value  1224.0   53.834355   24.070280  23.35   23.3500   57.20   
score             1224.0   58.710621   21.770931  30.85   30.8500   63.20   

                      75%     max  
rank              918.250  1224.0  
first_sdg_value    73.725    99.9  
second_sdg_value   71.875    99.9  
third_sdg_value    71.050    98.3  
fourth_sdg         17.000    17.0  
fourth_sdg_value   73.050    98.8  
score              74.700    99.4  

Median of each numeric column:
rank                612.50
first_sdg_value 

## 4. Bivariate/Multivariate Analysis:

Here, you look at the relationships between two or more variables. This can involve looking for correlations, patterns, and trends that suggest a relationship or an association.

We can perform bi-variate analysis for any combination of categorical and numerical variables. The combination can be:
| bi-variate variables   | Plot type |
| ------------- | ------------- |
| Categorical & Categorical| Stacked Bar Chart |
| Categorical & numerical  | scatter plot, histogram, box plot|
| numerical  & numerical  | Scatter plot, line chart| 


Multivariate Analysis:
- Heat map
- Bar Chart
- Scatter Chart
- Line Chart

**Categorical & Categorical --> (Stacked Column Chart)**

**Categorical & numerical --> (scatter plot, histogram, box plot)**

**numerical & numerical --> (Scatter plot, line chart)**

We could also use a correlation matrix to get more specific information about the relationship between these two variables.